In [8]:
import pandas as pd
import numpy as np
import pickle
from preprocess_text import *
from bigrams import *
from smoothers import *
from LanguageModels import *

In [9]:
df = pd.read_csv('A1_dataset.csv')

In [10]:
# text = df['TEXT'].to_list()
# def preprocess_text(text):
#     text = lowercase_text(text)
#     text = remove_url_html(text)
#     text = remove_users(text)
#     text = remove_punctuations(text)
#     text = remove_whitespaces(text)
#     text = tokenization(text)
#     text = spelling_correction(text, 'autocorrect_full')
#     text = remove_alphanum(text)
#     return text
# df['preprocessed_text'] = df['TEXT'].progress_apply(preprocess_text)
# preprocess_text = df['preprocessed_text'].to_list()
# df.to_csv('preprocessed_A1.csv', index=False)

In [11]:
df = pd.read_csv('preprocessed_A1.csv')
preprocess_text = df['preprocessed_text'].to_list()
for i in range(len(preprocess_text)):
    preprocess_text[i] = eval(preprocess_text[i])

In [12]:
unigram_counts = {}
for sentence in preprocess_text:
    for word in sentence:
        if word in unigram_counts:
            unigram_counts[word] += 1
        else:
            unigram_counts[word] = 1

In [13]:
unigram_probs = {}
scale_factor = sum(unigram_counts.values())
for word in unigram_counts:
    unigram_probs[word] = unigram_counts[word] / scale_factor

In [14]:
len(unigram_counts)

6952

# Saving Bigrams with Smoothing

In [15]:
# bigram_wrapper = Bigrams(preprocess_text)
# bigrams = bigram_wrapper.get_bigrams()
# vocab_len = bigram_wrapper.vocab_len

# with open('bigram_model_dumps/bigram_wrapper.pickle', 'wb') as f:
#     pickle.dump(bigram_wrapper, f)

In [16]:
# load bigram_wrapper
# with open('bigram_model_dumps/bigram_wrapper.pickle', 'rb') as f:
#     bigram_wrapper = pickle.load(f)

# bigrams = bigram_wrapper.get_bigrams()
# vocab_len = bigram_wrapper.vocab_len

In [17]:
# smoother = LMSmoothers(bigrams, vocab_len, unigram_counts=unigram_counts, unigrams_probs=unigram_probs)
# laplace_smoothed_bigrams = smoother.laplace()
# add_5_smoothed_bigrams = smoother.add_k(5)
# add_5_with_unigram_prior_smoothed_bigrams = smoother.add_k_with_unigram_prior(5)

In [18]:
# with open('bigram_model_dumps/laplace_smoothed_bigrams.pickle', 'wb') as handle:
#     pickle.dump(laplace_smoothed_bigrams, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('bigram_model_dumps/add_5_smoothed_bigrams.pickle', 'wb') as handle:
#     pickle.dump(add_5_smoothed_bigrams, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('bigram_model_dumps/add_5_with_unigram_prior_smoothed_bigrams.pickle', 'wb') as handle:
#     pickle.dump(add_5_with_unigram_prior_smoothed_bigrams, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Creating BiGram Models

In [19]:
with open('bigram_model_dumps/laplace_smoothed_bigrams.pickle', 'rb') as handle:
    laplace_smoothed_bigrams = pickle.load(handle)

# with open('bigram_model_dumps/add_5_smoothed_bigrams.pickle', 'rb') as handle:
#     add_5_smoothed_bigrams = pickle.load(handle)

# with open('bigram_model_dumps/add_5_with_unigram_prior_smoothed_bigrams.pickle', 'rb') as handle:
#     add_5_with_unigram_prior_smoothed_bigrams = pickle.load(handle)

In [20]:
# Load Positive Prompts
with open('Pos Neg Prompts/pos_250_hf.txt', 'rb') as handle:
    pos_prompts = handle.readlines()

for i in range(len(pos_prompts)):
    pos_prompts[i] = pos_prompts[i].decode('utf-8').strip()

In [21]:
# clear the file
open('pos_hf_prompts.txt', 'w').close()

In [22]:
def append_to_file(filename, text1, text2):
    with open(filename, 'a') as f:
        f.write(text1 + " " + text2 + '\n')

In [23]:
ls_pos_gen = []
lm = LanguageModel(laplace_smoothed_bigrams, unigram_probs, "vader", False, False, True, sentiment_scale_factor=1, normalizebyPerplexity=True)
for p in pos_prompts:
    sentence_pos = lm.generate_text([p], sentiment=1, length=10)
    print("Positive: {}".format(sentence_pos))
    ppl = lm.computePerplexity(sentence_pos)
    print("Positive Perplexity: {}".format(ppl))
    ls_pos_gen.append((sentence_pos, ppl))
    append_to_file('pos_hf_prompts.txt', " ".join(sentence_pos), str(ppl))

Positive: ['bliss', 'love', 'that', 'paradise', 'greatest', 'show', 'good', 'luck', 'freedom', 'perfectly']
Positive Perplexity: 3624.049332228823
Positive: ['happiness', 'if', 'it', 'is', 'good', 'luck', 'i', 'love', 'win', 'xp']
Positive Perplexity: 1182.5637140858719
Positive: ['enjoying', 'it', 'is', 'a', 'wonderful', 'time', 'to', 'win', 'xp', 'love']
Positive Perplexity: 1529.2448172424267
Positive: ['loved', 'ya', 'love', 'that', 'i', 'wish', 'freedom', 'of', 'greatest', 'show']
Positive Perplexity: 2296.046765555215
Positive: ['beautiful', 'piece', 'paradise', 'perfectly', 'then', 'love', 'the', 'great', 'day', 'awesome']
Positive Perplexity: 3508.2164909729163
Positive: ['enjoyed', 'it', 'is', 'great', 'weekend', 'freedom', 'of', 'paradise', 'perfectly', 'then']
Positive Perplexity: 3089.0972981284763
Positive: ['delighted', 'love', 'you', 'like', 'that', 'paradise', 'perfectly', 'freedom', 'glorious', 'sunshine']
Positive Perplexity: 2942.5976827672002
Positive: ['hilarious',